## 21 연산자 오버로드(2)

In [17]:
import weakref
import numpy as np
import contextlib


class Config:
    enable_backprop = True


@contextlib.contextmanager
def using_config(name, value):
    old_value = getattr(Config, name)
    setattr(Config, name, value)
    try:
        yield
    finally:
        setattr(Config, name, old_value)


def no_grad():
    return using_config('enable_backprop', False)


In [18]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [19]:
class Variable:
  def __init__(self, data, name = None) :
    if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))
    self.data = data
    # 이름 지정
    self.name = name
    self.grad = None
    self.creator = None
    #세대 수 표현
    self.generation = 0
  def set_creator(self, func):
    self.creator = func
    self.generation = func.generation + 1
  
  def backward(self, retain_grad = False):
    if self.grad is None:
      self.grad = np.ones_like(self.data)
    funcs = []
    seen_set = set()

    def add_func(f):
      if f not in seen_set:
        funcs.append(f)
        seen_set.add(f)
        funcs.sort(key = lambda x: x.generation)

    add_func(self.creator)
    while funcs:
      f = funcs.pop()
      # output이 많으니까 차례차례 진행
      gys = [output().grad for output in f.outputs]
      gxs = f.backward(*gys)
      if not isinstance(gxs, tuple) :
        # 튜플 변환
        gxs = (gxs,)
      for x, gx in zip(f.inputs, gxs):
        if x.grad is None:
          x.grad = gx
        else:
          x.grad = x.grad + gx
        if x.creator is not None:
          add_func(x.creator)
      # 중간 미분값 삭제
      if not retain_grad:
        for y in f.outputs:
          y().grad = None
  def cleargrad(self):
    self.grad = None
  #shape
  @property
  def shape(self):
    return self.data.shape
  
  @property
  def ndim(self):
    return self.data.ndim
  
  @property
  def size(self):
    return self.data.size
  #property는 메소드를 인스턴스 변수처럼 사용할 수 있음

  @property
  def dtype(self):
    return self.data.dtype
  
  def __len__(self):
    return len(self.data)
    
  def __repr__(self):
    if self.data is None:
      return 'variable(None)'
    p = str(self.data).replace('\n','\n'+' '*9)
    return 'variable(' + p + ')'


In [20]:
import weakref
class Function : 
  def __call__(self, *inputs):
    xs = [x.data for x in inputs]
    ys = self.forward(*xs)
    if not isinstance(ys, tuple):
      ys = (ys,)
    outputs = [Variable(as_array(y)) for y in ys ]
    if Config.enable_backprop:
      self.generation = max([x.generation for x in inputs])
      for output in outputs :
        output.set_creator(self)
      self.inputs = inputs
      self.outputs = [weakref.ref(output) for output in outputs]
    return outputs if len(outputs) > 1 else outputs[0]
  
  def forward(self,xs):
    raise NotImplementedError()

  def backward(self,gys):
    raise NotImplementedError()

In [21]:
class Add(Function):
  def forward(self,x0,x1):
    y = x0 + x1 
    return y
  def backward(self, gy):
    return gy,gy

In [22]:
def add(x0, x1):
  return Add()(x0,x1)

In [23]:
class Square(Function):
  def forward(self,x):
    y = x**2
    return y
    
  def backward(self, gy):
    x = self.inputs[0].data
    gx = 2*x*gy
    return gx

In [24]:
def square(x):
  return Square()(x)

In [25]:
class Mul(Function):
  def forward(self, x0, x1):
    y = x0 * x1
    return y
  
  def backward(self, gy):
    x0, x1 = self.inputs[0].data, self.inputs[1].data
    return gy * x1, gy * x0

In [26]:
def mul(x0, x1):
  return Mul()(x0,x1)

In [27]:
#굳이 함수를 쓰지 않고 직접 연결
Variable.__mul__ = mul
Variable.__add__ = add

### 21.1 ndarray와 함께 사용하기

+ ndarray 를 Variable로 자동으로 바꿔서 작업

In [28]:
def as_variable(obj):
  if isinstance(obj, Variable):
    return obj
  return Variable(obj)

In [30]:
import weakref
class Function : 
  def __call__(self, *inputs):
    inputs = [as_variable(x) for x in inputs]
    xs = [x.data for x in inputs]
    ys = self.forward(*xs)
    if not isinstance(ys, tuple):
      ys = (ys,)
    outputs = [Variable(as_array(y)) for y in ys ]
    if Config.enable_backprop:
      self.generation = max([x.generation for x in inputs])
      for output in outputs :
        output.set_creator(self)
      self.inputs = inputs
      self.outputs = [weakref.ref(output) for output in outputs]
    return outputs if len(outputs) > 1 else outputs[0]
  
  def forward(self,xs):
    raise NotImplementedError()

  def backward(self,gys):
    raise NotImplementedError()

In [31]:
class Add(Function):
  def forward(self,x0,x1):
    y = x0 + x1 
    return y
  def backward(self, gy):
    return gy,gy

In [32]:
def add(x0, x1):
  return Add()(x0,x1)

In [33]:
class Square(Function):
  def forward(self,x):
    y = x**2
    return y
    
  def backward(self, gy):
    x = self.inputs[0].data
    gx = 2*x*gy
    return gx

In [34]:
def square(x):
  return Square()(x)

In [35]:
class Mul(Function):
  def forward(self, x0, x1):
    y = x0 * x1
    return y
  
  def backward(self, gy):
    x0, x1 = self.inputs[0].data, self.inputs[1].data
    return gy * x1, gy * x0

In [36]:
def mul(x0, x1):
  return Mul()(x0,x1)

In [37]:
#굳이 함수를 쓰지 않고 직접 연결
Variable.__mul__ = mul
Variable.__add__ = add

In [38]:
x = Variable(np.array(2.0))
y = x + np.array(3.0)
print(y)

variable(5.0)


### 21.2 float, int 와 함께 사용하기
+ 각 함수에 as_array를 이용하여 바로 array로 만들고 다시 Function 들어갈 때 Variable로 바꿔줌

In [50]:
def add(x0, x1):
  x1 = as_array(x1)
  return Add()(x0, x1)

In [51]:
def mul(x0, x1):
  x1 = as_array(x1)
  return Mul()(x0,x1)

In [52]:
#굳이 함수를 쓰지 않고 직접 연결
Variable.__mul__ = mul
Variable.__add__ = add

In [53]:
x = Variable(np.array(2.0))
y = x + 3.0
print(y)

variable(5.0)


### 21.3 문제점 1 : 첫 번째 인수가 float이나 int인 경우

In [54]:
y = 2.0 * x

TypeError: ignored

+ 반대로 넣어주면 된다. (self에 Variable형식이 무조건 들어가야함)

In [55]:
Variable.__add__ = add
Variable.__radd__ = add
Variable.__mul__ = mul
Variable.__rmul__ = mul


In [56]:
x = Variable(np.array(2.0))
y = 3.0 * x + 1.0
print(y)

variable(7.0)


### 21.4 문제점 2 : 좌항이 ndarray 인스턴스인 경우


In [58]:
x = Variable(np.array([1.0]))
y = np.array([2.0]) + x
y

array([variable([3.])], dtype=object)

In [59]:
class Variable:
  __array_priority__ = 200
  def __init__(self, data, name = None) :
    if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))
    self.data = data
    # 이름 지정
    self.name = name
    self.grad = None
    self.creator = None
    #세대 수 표현
    self.generation = 0
  def set_creator(self, func):
    self.creator = func
    self.generation = func.generation + 1
  
  def backward(self, retain_grad = False):
    if self.grad is None:
      self.grad = np.ones_like(self.data)
    funcs = []
    seen_set = set()

    def add_func(f):
      if f not in seen_set:
        funcs.append(f)
        seen_set.add(f)
        funcs.sort(key = lambda x: x.generation)

    add_func(self.creator)
    while funcs:
      f = funcs.pop()
      # output이 많으니까 차례차례 진행
      gys = [output().grad for output in f.outputs]
      gxs = f.backward(*gys)
      if not isinstance(gxs, tuple) :
        # 튜플 변환
        gxs = (gxs,)
      for x, gx in zip(f.inputs, gxs):
        if x.grad is None:
          x.grad = gx
        else:
          x.grad = x.grad + gx
        if x.creator is not None:
          add_func(x.creator)
      # 중간 미분값 삭제
      if not retain_grad:
        for y in f.outputs:
          y().grad = None
  def cleargrad(self):
    self.grad = None
  #shape
  @property
  def shape(self):
    return self.data.shape
  
  @property
  def ndim(self):
    return self.data.ndim
  
  @property
  def size(self):
    return self.data.size
  #property는 메소드를 인스턴스 변수처럼 사용할 수 있음

  @property
  def dtype(self):
    return self.data.dtype
  
  def __len__(self):
    return len(self.data)
    
  def __repr__(self):
    if self.data is None:
      return 'variable(None)'
    p = str(self.data).replace('\n','\n'+' '*9)
    return 'variable(' + p + ')'


In [61]:
Variable.__add__ = add
Variable.__radd__ = add
Variable.__mul__ = mul
Variable.__rmul__ = mul


In [62]:
x = Variable(np.array([1.0]))
y = np.array([2.0]) + x
y

variable([3.])

+ np.array가 왼쪽에 있을 때 일반적으로 np.array 안의 mul 등이 실행되지만 __array_priority__ = 200
으로 연산자 우선순위를 높여버리면 오른쪽에 있어도 오른쪽 것을 채택한다.
